In [1]:
!pip install openpyxl lxml

In [2]:
import torch
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForTokenClassification,TrainingArguments, Trainer
import numpy as np
from tqdm.auto import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(1337)
torch.manual_seed(1337)

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_excel('CRA_train_1200.xlsx')

# Check the first few rows
print(df.head())

   Id                                             pr_txt Категория  \
0   1  Повышение кредитного рейтинга  Акционерного об...         A   
1   2  «Эксперт РА» подтвердил кредитный рейтинг комп...        BB   
2   3  НКР повысило кредитный рейтинг ООО "ОТЭКО-Порт...         A   
3   4  «Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...       AAA   
4   5  29 марта 2023 г. Ведущий рейтинговый аналитик ...       BBB   

  Уровень рейтинга  
0                A  
1               BB  
2                A  
3              AAA  
4              BBB  


In [4]:
mask = df['pr_txt'].str.len() > 200

# Use the boolean mask to select the rows that meet the condition
filtered_df = df[mask]

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize a label encoder
label_encoder = LabelEncoder()

# Encode the text labels
filtered_df['label'] = label_encoder.fit_transform(filtered_df['Уровень рейтинга'])

In [5]:
filtered_df['label'] = label_encoder.fit_transform(filtered_df['Категория'])

In [6]:
filtered_df['binary_label'] = 1-np.round(filtered_df['label']/6.0)

In [7]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(filtered_df, test_size=0.15, random_state=42)

# Reset index for both dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [8]:
from transformers import BertForMaskedLM,BertTokenizerFast, pipeline, BatchEncoding,BertForTokenClassification,BertForSequenceClassification
from tokenizers import Encoding
#model_ckpt = "sberbank-ai/ruBert-base"
model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizerFast.from_pretrained(model_ckpt)

In [10]:
# Define a custom dataset class

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset


class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=2048):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.data.loc[index, 'pr_txt'])
        label = int(self.data.loc[index, 'binary_label'])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

# Create DataLoader for training and validation
train_dataset = CustomDataset(train_df, tokenizer)
val_dataset = CustomDataset(val_df, tokenizer)

batch_size = 2

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)



In [11]:
from transformers import Trainer, TrainingArguments

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Define a function to compute F1 score
def compute_f1(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    return f1

# Modify the Trainer's compute_metrics function to include F1 score
def compute_metrics(p):
    preds = p.predictions
    labels = p.label_ids
    preds = preds.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds,average='weighted')
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [14]:

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir='./rubert-tiny-interp',
    evaluation_strategy='epoch',
    num_train_epochs=4,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    save_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    fp16 = True
)

# Create Trainer with modified compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Include F1 in metrics
)

# Start training
trainer.train()



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.333000,1.306388,0.804675,0.805556,0.805089
2,0.047600,1.352969,0.815841,0.816667,0.816227
3,0.026000,1.465680,0.800087,0.794444,0.796631
4,0.126600,1.447655,0.800087,0.794444,0.796631


TrainOutput(global_step=2040, training_loss=0.12348535855611166, metrics={'train_runtime': 293.6442, 'train_samples_per_second': 13.894, 'train_steps_per_second': 6.947, 'total_flos': 120347057848320.0, 'train_loss': 0.12348535855611166, 'epoch': 4.0})

In [16]:
trainer.evaluate()

{'eval_loss': 1.4476546049118042,
 'eval_precision': 0.8000865682522327,
 'eval_recall': 0.7944444444444444,
 'eval_f1': 0.7966305194276881,
 'eval_runtime': 4.1107,
 'eval_samples_per_second': 43.788,
 'eval_steps_per_second': 21.894,
 'epoch': 4.0}

In [9]:
from transformers import BertForMaskedLM,BertTokenizerFast, pipeline, BatchEncoding,BertForTokenClassification,BertForSequenceClassification

#model_ckpt = "sberbank-ai/sbert_large_nlu_ru"
model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizerFast.from_pretrained(model_ckpt)

model = BertForSequenceClassification.from_pretrained('./rubert-tiny-interp/checkpoint-2000/')

In [10]:
from transformers_interpret import SequenceClassificationExplainer

multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

In [18]:
word_attributions = multiclass_explainer(text=str("""Кредитный рейтинг ООО «Газпром экспорт» (далее — Компания, «Газпром экспорт») обусловлен очень сильной оценкой связей Компании с ПАО «Газпром», а также сильной оценкой категории поддерживающей организации (ПАО «Газпром» — <rating>, прогноз «Стабильный»). «Газпром экспорт» — сбытовая структура ПАО «Газпром». Принимая во внимание степень связей Компании с ПАО «Газпром», АКРА считает целесообразным использовать в рамках рейтингового анализа Компании только Методологию анализа взаимосвязи компаний внутри группы без определения оценки собственной кредитоспособности (ОСК) Компании. Ключевые факторы рейтинговой оценки Очень сильная оценка связи с поддерживающей организацией обусловлена очень сильной юридической связью Компании с ПАО «Газпром» (доля в капитале составляет 100%), полной операционной интеграцией, а также стратегической значимостью Компании для ПАО «Газпром». Реализация за пределы РФ продукции, производимой ПАО «Газпром» и его дочерними обществами, осуществляется централизованно через единый экспортный канал сбыта — «Газпром экспорт». Исключение составляет природный газ, поставляемый в страны СНГ и Балтии по контрактам, заключенным ПАО «Газпром». «Газпром экспорт» владеет сетью зарубежных сбытовых компаний (включая 100%-ную долю в холдинге «Газпром Германия»), осуществляющих маркетинг и торговлю углеводородами на европейском рынке, включая поставки газа конечным потребителям. «Газпром экспорт» также владеет долями в подземных хранилищах газа (ПХГ), имеющих стратегическое значение для ПАО «Газпром»: «Дамборжице» (Чехия), «Банатский Двор» (Сербия), «Катарина» (Германия), а через «Газпром Германия» — «Реден», «Этцель» и «Йемгум» в Германии, а также «Хайдах» в Австрии. Компания находится в системе физического кэш-пулинга ПАО «Газпром» и не осуществляет внешних заимствований. В соответствии с договорами комиссии ПАО «Газпром» возмещает Компании расходы, возникающие в связи с исполнением договоров комиссии, включая расходы на транспортировку и хранение газа. ПАО «Газпром» обеспечивает поддержание чистых активов Компании на достаточном уровне, в том числе путем внесения имущественного взноса в случае необходимости. Сильная оценка категории поддерживающей организации. В качестве поддерживающей организации Компании выступает ПАО «Газпром» — одна из крупнейших мировых интегрированных энергетических компаний. ПАО «Газпром» входит в число крупнейших производителей нефти в РФ (посредством ПАО «Газпром нефть» — <rating>, прогноз «Стабильный»). Кроме того, ПАО «Газпром» владеет крупными энергогенерирующими активами (через ООО «Газпром энергохолдинг»), а также газотранспортной системой протяженностью 171 тыс. км. Кредитный рейтинг ПАО «Газпром» обусловлен очень высокой системной значимостью организации для российской экономики и высокой степенью влияния на него со стороны государства (согласно методологии АКРА). Уровень собственной кредитоспособности ПАО «Газпром» определяется очень сильным бизнес-профилем, низким отраслевым риском и сильным финансовым профилем. Сдерживающее влияние на оценку собственной кредитоспособности ПАО «Газпром» оказывают средние показатели его долговой нагрузки и обслуживания долга, а также слабые показатели свободного денежного потока, обусловленные значительным объемом капитальных затрат и выплачиваемых дивидендов. Ключевые допущения сохранение акционерного и операционного контроля со стороны ПАО «Газпром». Факторы возможного изменения прогноза или рейтинга «Стабильный» прогноз предполагает с наиболее высокой долей вероятности неизменность рейтинга на горизонте 12–18 месяцев. К негативному рейтинговому действию может привести: потеря операционного или акционерного контроля со стороны ПАО «Газпром»; снижение кредитного рейтинга ПАО «Газпром». Компоненты рейтинга Оценка собственной кредитоспособности (ОСК): отсутствует. Корректировки: отсутствуют. Поддержка: паритет с ПАО «Газпром». Рейтинги выпусков Эмиссии в обращении отсутствуют. Регуляторное раскрытие Кредитный рейтинг был присвоен по национальной шкале для Российской Федерации на основе Методологии анализа взаимосвязи компаний внутри группы, а также Основных понятий, используемых Аналитическим Кредитным Рейтинговым Агентством в рейтинговой деятельности. Кредитный рейтинг был присвоен ООО «Газпром экспорт» впервые. Пересмотр кредитного рейтинга и прогноза по кредитному рейтингу ожидается в течение одного года с даты рейтингового действия (19.11.2018). Кредитный рейтинг был присвоен на основании данных, предоставленных ООО «Газпром экспорт», информации из открытых источников, а также баз данных АКРА. Кредитный рейтинг является запрошенным, ООО «Газпром экспорт» принимало участие в процессе присвоения кредитного рейтинга. Факты существенного отклонения информации от данных, зафиксированных в официально опубликованной финансовой отчетности ООО «Газпром экспорт», не выявлены. АКРА не оказывало ООО «Газпром экспорт» дополнительных услуг. Конфликты интересов в рамках процесса присвоения кредитного рейтинга выявлены не были."""))

In [12]:
word_attributions

[('[CLS]', 0.0),
 ('По', -0.011236784392196655),
 ('оценкам', 0.0018149358561473602),
 ('Агентства', 0.00828930935650628),
 (',', 0.007991110699249973),
 ('LE', 0.0542840508357214),
 ('##GE', -0.010030060942279736),
 ('##ND', -0.0022276361120843646),
 ('##A', 0.3199326355483814),
 ('является', -0.014856304471705243),
 ('успешным', 0.002982837045549503),
 ('игроком', -0.010665064762076761),
 ('на', 0.009900017202915638),
 ('первич', -0.012106401556335121),
 ('##ном', -0.000588310401383235),
 ('рынке', -0.0020189493791123608),
 ('жилья', 0.00018675472779941757),
 ('в', 0.012396300531909554),
 ('Санкт', 0.014605197725924987),
 ('-', 0.016223262252586593),
 ('Петербурге', 0.02597238513855007),
 ('.', 0.04213468945203399),
 ('По', 0.007939386435849896),
 ('итогам', 0.004196305577687342),
 ('первых', 0.006663711660242938),
 ('шести', 0.0016580336980851619),
 ('месяцев', 0.04540327502535885),
 ('2023', 0.03840327430363153),
 ('года', 0.03277388008978968),
 ('компания', -0.010563784090842736),

In [19]:
html = multiclass_explainer.visualize()

In [6]:
import eli5
from eli5.lime import TextExplainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/sklearn/utils/metaestimators.py)

In [22]:
!pip install Cython

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Obtaining dependency information for Cython from https://files.pythonhosted.org/packages/d9/fc/3a49ab49ae556b7d644a2d3082f01082bb902befe3e398f11d675d6c2ad0/Cython-3.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached Cython-3.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
Using cached Cython-3.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)


In [23]:
!pip install scikit-learn==0.21.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached scikit-learn-0.21.3.tar.gz (12.2 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [904 lines of output]
      /tmp/pip-install-ltmnkbk5/scikit-learn_246bc279a9a145be8cfb9043289395cc/setup.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import parse_version
      Partial import of sklearn during the build process.
      /tmp/pip-install-ltmnkbk5/scikit-learn_246bc279a9a145be8cfb9043289395cc/setup.py:122: DeprecationWarning:
      
    

In [17]:
# Define a sample text to explain
text_to_explain = "По оценкам Агентства, LEGENDA является успешным игроком"

# Tokenize the text
tokens = tokenizer(text_to_explain, return_tensors="pt", padding=True, truncation=True)

# Get the model's prediction
with torch.no_grad():
    logits = model(**tokens).logits

# Instantiate a TextExplainer
explainer = TextExplainer(random_state=42)

# Explain the model's prediction
explanation = explainer.explain_instance(
    text_to_explain,
    lambda x: model(**tokenizer(x, return_tensors="pt")).logits,
    labels=[1],  # Specify the label you want to explain (1 for positive class, 0 for negative)
)

# Display the explanation
eli5.show_weights(explanation)

NameError: name 'torch' is not defined